# 스윕 작업으로 하이퍼 매개 변수 조정

다수의 기계 학습 알고리즘에는 하이퍼 매개 변수가 필요합니다. 하이퍼 매개 변수란 학습에는 영향을 주지만 학습 데이터 자체에서는 확인할 수 없는 매개 변수 값입니다. 예를 들어 로지스틱 회귀 모델 학습 시에는 정규화 비율 하이퍼 매개 변수를 사용하여 모델의 바이어스를 완화할 수 있습니다. 그리고 콘볼루션 신경망 학습 시에는 학습 속도, 일괄 처리 크기 등의 하이퍼 매개 변수를 사용하여 가중치 조정 방식과 미니 배치에서 처리되는 데이터 항목의 수를 각각 제어할 수 있습니다. 선택하는 하이퍼 매개 변수 값에 따라 학습된 모델의 성능이나 모델을 학습시키는 데 걸리는 시간이 크게 달라질 수 있으며, 여러 매개 변수 조합을 적용하여 최적의 값을 찾아야 하는 경우가 많습니다. 

## 시작하기 전에

이 Notebook에서 코드를 실행하려면  **최신 버전의 azureml-ai-ml** 패키지가 필요합니다. 아래 셀의 명령을 실행하여 이 패키지가 설치되어 있는지 확인합니다.

> **고**:
> **azure-ai-ml** 패키지가 설치되지 않은 경우 를 실행 `pip install azure-ai-ml` 하여 설치합니다.

In [ ]:
## 작업 영역에 연결

필요한 SDK 패키지를 설치했으므로 작업 영역에 연결할 수 있습니다.

작업 영역에 연결하려면 구독 ID, 리소스 그룹 이름 및 작업 영역 이름 등 식별자 매개 변수가 필요합니다. 리소스 그룹 이름 및 작업 영역 이름이 이미 채워져 있습니다. 명령을 완료하려면 구독 ID만 필요합니다.

필요한 매개 변수를 찾으려면 Studio의 오른쪽 위에 있는 구독 및 작업 영역 이름을 클릭합니다. 오른쪽에 창이 열립니다.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 구독 ID를 복사하고 **YOUR-SUBSCRIPTION-ID** 를 복사한 값으로 바꿉니다. </p>

## 학습 스크립트 만들기
하이퍼 매개 변수 튜닝은 기계 학습 모델을 학습시키고 입력 매개 변수를 변경하려는 경우에 이상적입니다. 알고리즘의 하이퍼 매개 변수 중 하나를 나타내는 입력 매개 변수가 필요한 학습 스크립트를 만들어야 합니다.

다음 셀을 실행하여 **src** 폴더 및 학습 스크립트를 만듭니다.

학습 스크립트에는 다음 두 개의 입력 매개 변수가 필요합니다.

- `--training_data` 문자열이 필요한 경우 등록된 데이터 자산의 경로를 입력 학습 데이터로 지정합니다.
- `--reg_rate` 숫자를 예상하지만 기본값 `0.01`은 입니다. 하이퍼 매개 변수 튜닝에 이 입력 매개 변수를 사용합니다.

In [ ]:
## 명령 작업 구성 및 실행

아래 셀을 실행하여 당뇨병을 예측하는 분류 모델을 학습시킵니다. 모델은 **src** 폴더에서 찾을 수 있는 **학습\.py** 스크립트를 실행하여 학습됩니다. 등록된 `diabetes-data` 데이터 자산을 학습 데이터로 사용합니다. 

- `code`: 실행할 스크립트를 포함하는 폴더를 지정합니다.
- `command`: 정확히 실행할 항목을 지정합니다.
- `environment`: 명령을 실행하기 전에 컴퓨팅에 설치해야 하는 패키지를 지정합니다.
- `compute`: 명령을 실행하는 데 사용할 컴퓨팅을 지정합니다.
- `display_name`: 개별 작업의 이름입니다.
- `experiment_name`: 작업이 속한 실험의 이름입니다.

명령 작업은 정규화 속도로 `0.1`학습 스크립트를 한 번만 실행합니다. 스윕 작업을 실행하여 하이퍼 매개 변수를 튜닝하기 전에 스크립트가 명령 작업에서 예상대로 작동하는지 테스트하는 것이 좋습니다.

In [ ]:
## 검색 공간 정의

명령 작업이 성공적으로 완료되면 스윕 작업을 구성하고 실행할 수 있습니다. 

먼저 하이퍼 매개 변수의 검색 공간을 지정해야 합니다. 각각 다른 정규화 속도(`0.01`, `0.1`또는 `1`)를 사용하여 세 가지 모델을 학습하려면 하이퍼 매개 변수를 사용하여 `Choice` 검색 공간을 정의할 수 있습니다. 

## 스윕 작업 구성 및 제출

스윕 함수를 사용하여 학습 스크립트에서 하이퍼 매개 변수 튜닝을 수행합니다. 스윕 작업을 구성하려면 다음을 구성해야 합니다.

- `compute`: 작업을 실행할 컴퓨팅 대상의 이름입니다.
- `sampling_algorithm`: 검색 공간에서 사용할 하이퍼 매개 변수 샘플링 알고리즘입니다. 허용되는 값은 `random`, `grid` 및 `bayesian`입니다.
- `primary_metric`: 각 평가판 작업에서 보고하는 기본 메트릭의 이름입니다. 메트릭은 동일한 메트릭 이름으로 `mlflow.log_metric()`을 사용하여 사용자의 학습 스크립트에 로그되어야 합니다.
- `goal`: 의 최적화 목표입니다 `primary_metric`. 허용되는 값은 `maximize` 및 `minimize`입니다.
- `limits`: 스윕 작업에 대한 제한입니다. 예를 들어 학습하려는 최대 평가판 또는 모델 양입니다.

명령 작업은 스윕 작업의 기본으로 사용됩니다. 명령 작업에 대한 구성은 스윕 작업에서 다시 사용합니다.

In [ ]:
다음 셀을 실행하여 스윕 작업을 제출합니다.

In [ ]:
작업이 완료되면 작업 개요로 이동합니다. **평가판** 탭에는 학습된 모든 모델과 시도한 각 정규화 속도 값에 대한 점수가 어떻게 다른지 `Accuracy` 표시됩니다.

## Configure and run a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train\.py** script that can be found in the **src** folder. It uses the registered `diabetes-data` data asset as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command job only runs the training script once, with a regularization rate of `0.1`. Before you run a sweep job to tune hyperparameters, it's a best practice to test whether your script works as expected with a command job.

In [ ]:
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes

# configure job

job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:diabetes-data:1"
            ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-mlflow",
    experiment_name="diabetes-training", 
    tags={"model_type": "LogisticRegression"}
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

## Define the search space

When your command job has completed successfully, you can configure and run a sweep job. 

First, you'll need to specify the search space for your hyperparameter. To train three models, each with a different regularization rate (`0.01`, `0.1`, or `1`), you can define the search space with a `Choice` hyperparameter. 

In [ ]:
from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

## Configure and submit the sweep job

You'll use the sweep function to do hyperparameter tuning on your training script. To configure a sweep job, you'll need to configure the following:

- `compute`: Name of the compute target to execute the job on.
- `sampling_algorithm`: The hyperparameter sampling algorithm to use over the search space. Allowed values are `random`, `grid` and `bayesian`.
- `primary_metric`: The name of the primary metric reported by each trial job. The metric must be logged in the user's training script using `mlflow.log_metric()` with the same corresponding metric name.
- `goal`: The optimization goal of the `primary_metric`. The allowed values are `maximize` and `minimize`.
- `limits`: Limits for the sweep job. For example, the maximum amount of trials or models you want to train.

Note that the command job is used as the base for the sweep job. The configuration for the command job will be reused by the sweep job.

In [ ]:
# apply the sweep parameter to obtain the sweep_job
sweep_job = command_job_for_sweep.sweep(
    compute="aml-cluster",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)

# set the name of the sweep job experiment
sweep_job.experiment_name="sweep-diabetes"

# define the limits for this sweep
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)

Run the following cell to submit the sweep job.

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)

When the job is completed, navigate to the job overview. The **Trials** tab will show all models that have been trained and how the `Accuracy` score differs for each regularization rate value you tried.